# Jour 3 - Exercice 3 : Régression pour la Prédiction des Prix de Vols

## Objectifs
- Appliquer les techniques de machine learning à un problème de régression réel
- Utiliser les données de prix de vols pour prédire le prix d'un billet d'avion
- Évaluer les performances des modèles sur un jeu de test
- Visualiser et interpréter les résultats avec Plotly

## Introduction

Dans ce notebook, nous allons travailler sur un problème de régression : prédire le prix d'un billet d'avion en fonction de différentes caractéristiques du vol. Nous utiliserons les données de prix de vols et appliquerons différentes techniques de machine learning pour construire et évaluer des modèles de régression.

## 1. Chargement et exploration des données

In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import pickle
import sys
import os
from datetime import datetime

# Visualisation avec Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scikit-learn pour le machine learning
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Modèles de régression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

# Ajouter le chemin pour importer utils.py
sys.path.append(os.path.abspath('./'))
from utils import results_predictions_prices

# Pour afficher plus de colonnes dans les DataFrames
pd.set_option('display.max_columns', 30)

In [ ]:
# Chargement des données d'entraînement
train_df = pd.read_csv('../../data/flight_prices/train.csv')

# Affichage des premières lignes
train_df.head()

In [ ]:
# Informations sur le DataFrame
train_df.info()

In [ ]:
# Statistiques descriptives
train_df.describe()

In [ ]:
# Vérification des valeurs manquantes
missing_values = train_df.isnull().sum()
missing_percentage = (missing_values / len(train_df)) * 100

missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

# Affichage des valeurs manquantes
missing_df[missing_df['Missing Values'] > 0].sort_values('Missing Values', ascending=False)

## 2. Analyse exploratoire des données

In [ ]:
# Distribution de la variable cible (Prix)
fig = px.histogram(train_df, x='Price', nbins=50, title='Distribution des Prix des Vols')
fig.update_layout(xaxis_title='Prix', yaxis_title='Fréquence')
fig.show()

In [7]:
# Conversion des dates en format datetime
train_df['Searched Date'] = pd.to_datetime(train_df['Searched Date'])
train_df['Departure Date'] = pd.to_datetime(train_df['Departure Date'])
train_df['Arrival Date'] = pd.to_datetime(train_df['Arrival Date'])

# Extraction de caractéristiques temporelles
train_df['Search_Month'] = train_df['Searched Date'].dt.month
train_df['Search_Day'] = train_df['Searched Date'].dt.day
train_df['Search_DayOfWeek'] = train_df['Searched Date'].dt.dayofweek

train_df['Departure_Month'] = train_df['Departure Date'].dt.month
train_df['Departure_Day'] = train_df['Departure Date'].dt.day
train_df['Departure_DayOfWeek'] = train_df['Departure Date'].dt.dayofweek
train_df['Departure_Hour'] = train_df['Departure Date'].dt.hour

train_df['Arrival_Month'] = train_df['Arrival Date'].dt.month
train_df['Arrival_Day'] = train_df['Arrival Date'].dt.day
train_df['Arrival_DayOfWeek'] = train_df['Arrival Date'].dt.dayofweek
train_df['Arrival_Hour'] = train_df['Arrival Date'].dt.hour

# Calcul de la durée du vol en heures
train_df['Flight_Duration_Hours'] = (train_df['Arrival Date'] - train_df['Departure Date']).dt.total_seconds() / 3600

# Calcul du nombre de jours entre la recherche et le départ
train_df['Days_Before_Departure'] = (train_df['Departure Date'] - train_df['Searched Date']).dt.days

In [ ]:
# Visualisation du prix en fonction de la durée du vol
fig = px.scatter(train_df, x='Flight_Duration_Hours', y='Price', color='Cabin',
                 title='Prix en fonction de la Durée du Vol',
                 labels={'Flight_Duration_Hours': 'Durée du Vol (heures)', 'Price': 'Prix'})
fig.show()

In [ ]:
# Visualisation du prix en fonction du nombre de jours avant le départ
fig = px.scatter(train_df, x='Days_Before_Departure', y='Price', color='Cabin',
                 title='Prix en fonction du Nombre de Jours avant le Départ',
                 labels={'Days_Before_Departure': 'Jours avant le Départ', 'Price': 'Prix'})
fig.show()

In [ ]:
# Prix moyen par compagnie aérienne
airline_price = train_df.groupby('Airline')['Price'].mean().reset_index().sort_values('Price', ascending=False)

fig = px.bar(airline_price, x='Airline', y='Price',
             title='Prix Moyen par Compagnie Aérienne',
             labels={'Airline': 'Compagnie Aérienne', 'Price': 'Prix Moyen'})
fig.show()

In [ ]:
# Prix moyen par classe de cabine
cabin_price = train_df.groupby('Cabin')['Price'].mean().reset_index().sort_values('Price', ascending=False)

fig = px.bar(cabin_price, x='Cabin', y='Price',
             title='Prix Moyen par Classe de Cabine',
             labels={'Cabin': 'Classe de Cabine', 'Price': 'Prix Moyen'})
fig.show()

In [ ]:
# Prix moyen par nombre d'escales
stops_price = train_df.groupby('Number Of Stops')['Price'].mean().reset_index().sort_values('Number Of Stops')

fig = px.bar(stops_price, x='Number Of Stops', y='Price',
             title='Prix Moyen par Nombre d\'Escales',
             labels={'Number Of Stops': 'Nombre d\'Escales', 'Price': 'Prix Moyen'})
fig.show()

## 3. Préparation des données pour la modélisation

In [13]:
# Calcul de la durée du vol en heures
train_df['Flight_Duration_Hours'] = (train_df['Arrival Date'] - train_df['Departure Date']).dt.total_seconds() / 3600

# Calcul du nombre de jours entre la recherche et le départ
train_df['Days_Before_Departure'] = (train_df['Departure Date'] - train_df['Searched Date']).dt.days

In [14]:
# Sélection des caractéristiques pour la modélisation
# Nous excluons les colonnes de date brutes et utilisons les caractéristiques extraites
features = [
    'Departure Airport', 'Arrival Airport', 'Number Of Stops', 'Airline', 'Cabin',
    'Flight Lands Next Day', 'Search_Month', 'Search_Day', 'Search_DayOfWeek',
    'Departure_Month', 'Departure_Day', 'Departure_DayOfWeek', 'Departure_Hour',
    'Arrival_Month', 'Arrival_Day', 'Arrival_DayOfWeek', 'Arrival_Hour',
    'Flight_Duration_Hours', 'Days_Before_Departure'
]

# Variable cible
target = 'Price'

# Séparation des caractéristiques et de la cible
X = train_df[features]
y = train_df[target]

# Séparation des données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Identification des colonnes numériques et catégorielles
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()

print(f"Caractéristiques numériques: {numeric_features}")
print(f"Caractéristiques catégorielles: {categorical_features}")

In [ ]:
# Création d'un préprocesseur pour les données
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Préparation des données d'entraînement et de validation
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

print(f"Forme des données d'entraînement transformées: {X_train_processed.shape}")
print(f"Forme des données de validation transformées: {X_val_processed.shape}")

## 4. Modélisation et évaluation

In [17]:
# Fonction pour évaluer les modèles
def evaluate_model(model, X_train, y_train, X_val, y_val):
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédictions sur l'ensemble de validation
    y_pred = model.predict(X_val)
    
    # Calcul des métriques
    mae = mean_absolute_error(y_val, y_pred)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, y_pred)
    
    return {
        'model': model.__class__.__name__,
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'model_object': model
    }

In [ ]:
# Création des modèles
models = [
    LinearRegression(),
    Ridge(alpha=1.0),
    Lasso(alpha=0.2),
    ElasticNet(alpha=0.1, l1_ratio=0.5),
    DecisionTreeRegressor(max_depth=10, random_state=42),
    RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42),
    GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, max_depth=3, random_state=42)
]

# Évaluation des modèles
results = []
for model in models:
    print("Training model: ", model.__class__.__name__)
    result = evaluate_model(model, X_train_processed, y_train, X_val_processed, y_val)
    results.append(result)
    
# Création d'un DataFrame avec les résultats
results_df = pd.DataFrame(results)
results_df = results_df.drop('model_object', axis=1)
results_df.sort_values('mae')

In [ ]:
# Visualisation des résultats
fig = px.bar(results_df, x='model', y='mae', title='Erreur Absolue Moyenne (MAE) par Modèle',
             labels={'model': 'Modèle', 'mae': 'MAE'},
             color='mae', color_continuous_scale='Viridis')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
# Visualisation des résultats (R²)
fig = px.bar(results_df, x='model', y='r2', title='Coefficient de Détermination (R²) par Modèle',
             labels={'model': 'Modèle', 'r2': 'R²'},
             color='r2', color_continuous_scale='Viridis')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
# Sélection du meilleur modèle (basé sur MAE)
best_model_idx = results_df['mae'].idxmin()
best_model_name = results_df.loc[best_model_idx, 'model']
best_model = results[best_model_idx]['model_object']

print(f"Le meilleur modèle est {best_model_name} avec une MAE de {results_df.loc[best_model_idx, 'mae']:.2f}")

In [ ]:
# Visualisation des prédictions vs valeurs réelles pour le meilleur modèle
y_pred = best_model.predict(X_val_processed)

fig = px.scatter(x=y_val, y=y_pred, title=f'Valeurs Réelles vs Prédictions ({best_model_name})',
                 labels={'x': 'Valeurs Réelles', 'y': 'Prédictions'})

# Ajout de la ligne d'identité (y=x)
fig.add_trace(go.Scatter(x=[y_val.min(), y_val.max()], y=[y_val.min(), y_val.max()],
                         mode='lines', name='y=x', line=dict(color='red', dash='dash')))

fig.show()

In [ ]:
# Visualisation de la distribution des erreurs
errors = y_val - y_pred

fig = px.histogram(errors, nbins=50, title='Distribution des Erreurs de Prédiction',
                   labels={'value': 'Erreur', 'count': 'Fréquence'})
fig.show()

## 5. Évaluation sur le jeu de test

In [ ]:
# Chargement des données d'entraînement et de test
train_df = pd.read_csv('../../data/flight_prices/train.csv')
test_df = pd.read_csv('../../data/flight_prices/test.csv')

# Affichage des premières lignes du jeu de test
test_df.head()

In [25]:
# Fonction pour préparer les données
def prepare_data(df):
    # Conversion des dates en format datetime
    df['Searched Date'] = pd.to_datetime(df['Searched Date'])
    df['Departure Date'] = pd.to_datetime(df['Departure Date'])
    df['Arrival Date'] = pd.to_datetime(df['Arrival Date'])
    
    # Extraction de caractéristiques temporelles
    df['Search_Month'] = df['Searched Date'].dt.month
    df['Search_Day'] = df['Searched Date'].dt.day
    df['Search_DayOfWeek'] = df['Searched Date'].dt.dayofweek
    
    df['Departure_Month'] = df['Departure Date'].dt.month
    df['Departure_Day'] = df['Departure Date'].dt.day
    df['Departure_DayOfWeek'] = df['Departure Date'].dt.dayofweek
    df['Departure_Hour'] = df['Departure Date'].dt.hour
    
    df['Arrival_Month'] = df['Arrival Date'].dt.month
    df['Arrival_Day'] = df['Arrival Date'].dt.day
    df['Arrival_DayOfWeek'] = df['Arrival Date'].dt.dayofweek
    df['Arrival_Hour'] = df['Arrival Date'].dt.hour
    
    # Calcul de la durée du vol en heures
    df['Flight_Duration_Hours'] = (df['Arrival Date'] - df['Departure Date']).dt.total_seconds() / 3600
    
    # Calcul du nombre de jours entre la recherche et le départ
    df['Days_Before_Departure'] = (df['Departure Date'] - df['Searched Date']).dt.days
    
    return df

In [26]:
# Préparation des données d'entraînement et de test
train_df = prepare_data(train_df)
test_df = prepare_data(test_df)

In [27]:
# Sélection des caractéristiques pour la modélisation
features = [
    'Departure Airport', 'Arrival Airport', 'Number Of Stops', 'Airline', 'Cabin',
    'Flight Lands Next Day', 'Search_Month', 'Search_Day', 'Search_DayOfWeek',
    'Departure_Month', 'Departure_Day', 'Departure_DayOfWeek', 'Departure_Hour',
    'Arrival_Month', 'Arrival_Day', 'Arrival_DayOfWeek', 'Arrival_Hour',
    'Flight_Duration_Hours', 'Days_Before_Departure'
]

# Variable cible
target = 'Price'

# Séparation des caractéristiques et de la cible pour l'entraînement
X_train = train_df[features]
y_train = train_df[target]

# Préparation des données de test
X_test = test_df[features]

In [28]:
# Identification des colonnes numériques et catégorielles
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'bool']).columns.tolist()

# Création d'un préprocesseur pour les données
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# Entraînement des modèles sur l'ensemble complet d'entraînement
# Nous allons entraîner plusieurs modèles et les évaluer sur le jeu de test

# Préparation des données d'entraînement
X_train_processed = preprocessor.fit_transform(X_train)

# Préparation des données de test
X_test_processed = preprocessor.transform(X_test)

print(f"Forme des données d'entraînement transformées: {X_train_processed.shape}")
print(f"Forme des données de test transformées: {X_test_processed.shape}")

In [ ]:
# Création et entraînement des modèles
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.2),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
}

# Entraînement des modèles
for name, model in models.items():
    print(f"Entraînement du modèle {name}...")
    model.fit(X_train_processed, y_train)
    print(f"Modèle {name} entraîné avec succès.")

In [ ]:
# Prédictions sur le jeu de test avec le modèle Gradient Boosting
# (généralement l'un des plus performants pour ce type de problème)
best_model = models['Gradient Boosting']
y_pred_test = best_model.predict(X_test_processed)

# Affichage des premières prédictions
pd.DataFrame({'Prédictions': y_pred_test}).head(10)

In [ ]:
# Évaluation des performances sur le jeu de test en utilisant la fonction results_predictions_prices
mae_test = results_predictions_prices(y_pred_test)
print(f"Erreur Absolue Moyenne (MAE) sur le jeu de test: {mae_test:.2f}")

## 6. Comparaison des performances des différents modèles sur le jeu de test

In [ ]:
# Évaluation de tous les modèles sur le jeu de test
test_results = []

for name, model in models.items():
    # Prédictions sur le jeu de test
    y_pred = model.predict(X_test_processed)
    
    # Calcul de la MAE en utilisant la fonction results_predictions_prices
    mae = results_predictions_prices(y_pred)
    
    test_results.append({
        'model': name,
        'mae': mae
    })
    
# Création d'un DataFrame avec les résultats
test_results_df = pd.DataFrame(test_results)
test_results_df.sort_values('mae')

In [ ]:
# Visualisation des résultats sur le jeu de test
fig = px.bar(test_results_df, x='model', y='mae', title='Erreur Absolue Moyenne (MAE) par Modèle sur le Jeu de Test',
             labels={'model': 'Modèle', 'mae': 'MAE'},
             color='mae', color_continuous_scale='Viridis')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

## 7. Analyse des caractéristiques importantes (pour le modèle Random Forest)

In [ ]:
# Extraction des noms des caractéristiques après transformation
feature_names = []

# Pour les caractéristiques numériques
feature_names.extend(numeric_features)

# Pour les caractéristiques catégorielles, nous devons extraire les noms des colonnes après one-hot encoding
# Nous allons créer une liste temporaire pour simuler les noms
categorical_feature_names = []
for feature in categorical_features:
    unique_values = X_train[feature].unique()
    for value in unique_values:
        categorical_feature_names.append(f"{feature}_{value}")

feature_names.extend(categorical_feature_names)

# Analyse des caractéristiques importantes pour le modèle Random Forest
rf_model = models['Random Forest']

# Extraction des importances des caractéristiques
importances = {}

# Pour les caractéristiques numériques
for i, feature in enumerate(numeric_features):
    importances[feature] = rf_model.feature_importances_[i]

# Pour les caractéristiques catégorielles
start_idx = len(numeric_features)
for feature in categorical_features:
    unique_values = X_train[feature].unique()
    feature_importance = 0
    for value in unique_values:
        feature_importance += rf_model.feature_importances_[start_idx]
        start_idx += 1
    importances[feature] = feature_importance

# Création d'un DataFrame pour visualiser les importances
importances_df = pd.DataFrame({
    'Feature': list(importances.keys()),
    'Importance': list(importances.values())
}).sort_values('Importance', ascending=False)

# Affichage des 10 caractéristiques les plus importantes
importances_df.head(10)

In [ ]:
# Visualisation des importances des caractéristiques
fig = px.bar(importances_df.head(10), x='Importance', y='Feature', orientation='h',
             title='Top 10 des Caractéristiques les Plus Importantes (Random Forest)',
             labels={'Importance': 'Importance', 'Feature': 'Caractéristique'},
             color='Importance', color_continuous_scale='Viridis')
fig.show()

## 8. Conclusion

Dans ce notebook, nous avons exploré un problème de régression pour prédire le prix des vols en fonction de différentes caractéristiques. Nous avons :

1. Chargé et exploré les données de prix de vols
2. Effectué une analyse exploratoire des données pour comprendre les relations entre les variables
3. Préparé les données pour la modélisation en extrayant des caractéristiques pertinentes
4. Entraîné plusieurs modèles de régression et évalué leurs performances
5. Utilisé le jeu de test pour évaluer les performances finales des modèles
6. Analysé les caractéristiques les plus importantes pour la prédiction des prix

Les résultats montrent que les modèles d'ensemble comme le Random Forest et le Gradient Boosting sont les plus performants pour ce problème de régression. Les caractéristiques les plus importantes pour la prédiction des prix incluent la durée du vol, le nombre de jours avant le départ, et la classe de cabine.

Pour améliorer davantage les performances, nous pourrions :
- Effectuer une optimisation des hyperparamètres plus poussée
- Explorer d'autres techniques de feature engineering
- Essayer des modèles plus avancés comme les réseaux de neurones
- Combiner plusieurs modèles en utilisant des techniques d'ensemble plus sophistiquées